In [2]:
# Check device number
!ls -ltrh /dev/video*

crw-rw---- 1 root video 81, 0 Jul 28 02:13 /dev/video0


In [3]:
from jetcam.usb_camera import USBCamera
from jetcam.csi_camera import CSICamera

# for USB Camera (Logitech C270 webcam), uncomment the following line
camera = USBCamera(width=224, height=224, capture_device=0) # confirm the capture_device number

# for CSI Camera (Raspberry Pi Camera Module V2), uncomment the following line
# camera = CSICamera(width=224, height=224, capture_device=0) # confirm the capture_device number

camera.running = True
print("camera created")

camera created


In [4]:
PATH = 'sll-more.pth'

In [5]:
PATH = 'sll-less.pth'

In [6]:
import torch
import torchvision.models as models

model = models.alexnet(pretrained=True)
print('model init')

#change last layer to number of categories (27 for 26 letters + empty)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 27)

#cuda
device = torch.device('cuda')
model = model.to(device)
model.load_state_dict(torch.load(PATH))
print('model loaded')

model init
model loaded


In [31]:
# game
import ipywidgets
import torch.nn.functional as F
import torch
import torchvision.transforms as transforms
import PIL.Image
import numpy as np
import traitlets
from traitlets import HasTraits, List, signature_has_traits
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
import threading
import cv2

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()

def preprocess(image):
    global device
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

LABELS = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'empty']
mode_widget = ipywidgets.ToggleButtons(options=['Spelling', 'Guessing', 'Random Guessing'], description='Game Mode', value='Spelling')
rule_widget = ipywidgets.Textarea(description='Rule', disabled=True, value="'Current Word' will display a word from dictionary\nYou will need to spell each letter of that word using the ASL Alphabet\nYou will have 3 seconds for each letter\nWrong spelling will end the game\nThe word will get longer as time goes on")
current_word_widget = ipywidgets.Text(description='Current Word', placeholder='Selected word goes here')
timer_widget = ipywidgets.IntText(description='Timer', placeholder='Countdown')
prediction_widget = ipywidgets.Text(description='Prediction', placeholder='Detected letter goes here')
score_widget = ipywidgets.IntText(description='Score', placeholder='Your score')
state_widget = ipywidgets.ToggleButtons(options=['Start', 'Stop'], description='Game State', value='Stop')

camera_widget = ipywidgets.Image()
# traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

def live(state_widget, model, camera, prediction_widget, score_widget):
    print(state_widget.value)
    global LABELS
    while state_widget.value == 'Start':
        image = camera.value
        preprocessed = preprocess(image)
        output = model(preprocessed)
        probs = F.softmax(output, dim=1).detach().cpu().numpy().flatten()
        category_index = probs.argmax()
        prediction_widget.value = LABELS[category_index]
#         score_widget.value = str(probs.max())
            
def start_game(change):
    # first time starting model will take a while before model can start predicting
    if change['new'] == 'Start':
        execute_thread = threading.Thread(target=live, args=(state_widget, model, camera, prediction_widget, score_widget))
        execute_thread.start()
        
def change_game_mode(change):
    global rule_widget, current_word_widget, timer_widget, prediction_widget, score_widget
    if change['new'] == 'Spelling':
        rule_widget.value = "'Current Word' will display a word from dictionary\nYou will need to spell each letter of that word using the ASL Alphabet\nYou will have 3 seconds for each letter\nWrong spelling will end the game\nThe word will get longer as time goes on"
    elif change['new'] == 'Guessing':
        rule_widget.value = "'Current Word' will display a word from dictionary\nYou will have 5 seconds to memorize the word\nAfter that 'Current Word' will display the position of the letter you need to guess and you will have 5 seconds to guess the letter\nWrong guess will end the game\nThe word will get longer as time goes on"
    elif change['new'] == 'Random Guessing':
        rule_widget.value = "'Current Word' will display a random string of letters\nYou will have 5 seconds to memorize the string\nAfter that 'Current Word' will display the position of the letter you need to guess and you will have 5 seconds to guess the letter\nWrong guess will end the game\nThe word will get longer as time goes on"
    current_word_widget.value = ''
    timer_widget.value = 0
    prediction_widget.value = ''
    score_widget.value = 0
        
mode_widget.observe(change_game_mode, names='value')
state_widget.observe(start_game, names='value')

live_execution_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget, prediction_widget]),
    mode_widget,
    rule_widget,
    current_word_widget,
    timer_widget,
    score_widget,
    state_widget
])

print("widgets created")

widgets created


In [36]:
# link camera
camera.unobserve_all()
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

In [37]:
all_widget = ipywidgets.VBox([
    ipywidgets.HBox([live_execution_widget])
])

display(all_widget)